# **Settings**

In [ ]:
# 1. mount Google drive
from google.colab import drive
drive.mount('/content/drive')

# 2. add the path to Colab VM
import sys
import os

FOLDERNAME= 'My Drive/Colab Notebooks/lightning_sandbox/lightning_sandbox'
assert FOLDERNAME is not None, "[!] Enter the foldername."
os.chdir(f"/content/drive/{FOLDERNAME}")
sys.path.append(os.getcwd())

Mounted at /content/drive


In [ ]:
# auto reload imported objects
%load_ext autoreload
%autoreload 2

In [ ]:
# set pytorch-lightning
!pip install pytorch-lightning --quiet
!pip install jsonargparse[signatures] --quiet

In [ ]:
import torch
import pytorch_lightning as pl

# **Execute examples**

In [ ]:
from data_module.CIFAR_IO import CIFAR_IO

PATH_DATASETS = '/content/data'
BATCH_SIZE = 64
NUM_WORKERS = 1

myData = CIFAR_IO(PATH_DATASETS, BATCH_SIZE, NUM_WORKERS)

In [ ]:
from module.AutoEncoder import AutoEncoder

LEARNING_RATE = 1e-3
OUTPUT_DIM = 128

myModel = AutoEncoder(LEARNING_RATE, OUTPUT_DIM)

In [ ]:
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint

trainer = pl.Trainer(
    max_epochs=100,
    callbacks=[
        ModelCheckpoint(monitor='val_loss', mode='max'),
        EarlyStopping(monitor='val_loss', mode='max', patience=3)
        ],

    # debugging #
    # overfit_batches=0.1,
    # fast_dev_run=True,

    # auto initializing #
    # auto_lr_find=True,
    # auto_scale_batch_size="power",
    )

In [ ]:
trainer.tune(myModel, myData)

In [ ]:
trainer.fit(myModel, myData)

# **Logger**

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir=logs/test_logs
# --default_hp_metric=False


# **CLI Notes**

In [ ]:

# cd to lightning_sandbox
'''
cd drive/My\ Drive/Colab\ Notebooks/lightning_sandbox/lightning_sandbox
'''

# fit configuration
'''
python main.py fit --model AutoEncoder\
 --config fit.yaml\
 --trainer.accelerator "gpu"\
 --trainer.devices 1\
 multiple --model.learning_rate "uniform(7.3e-5, 9.3e-5, 4)"
'''

# test configuration
'''
python main.py test --model AutoEncoder\
 --config test.yaml\
 --ckpt_path fit_logs/version_5/checkpoints/epoch=35-step=27576.ckpt
'''